<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_02_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

url_pur = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_purchase_log.csv'
url_cus = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_customer_mst.csv'
url_pro = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/ds_product_mst.csv'

data_pur = pd.read_csv(url_pur)
data_cus = pd.read_csv(url_cus)
data_pro = pd.read_csv(url_pro)

In [0]:
# [0] aggregation - category values into one list item

import numpy as np

# if cust_no & update_dt is dup, get max age_gender
cus_group = data_cus.groupby(['cust_no', 'update_dt']).agg({'age_gender': 'max'})

# sort by cust_no ascending, by update_dt descending to get latest age_gender
cus_group = cus_group.sort_values(by=['cust_no', 'update_dt'],
                                  ascending=[True, False])
cus_group.reset_index(inplace=True)
# groupby cust_no and put age_gender in a list
cus_group = cus_group.groupby('cust_no').agg({'age_gender': lambda x: list(x)})
# get the first age_gender in the list
cus_group = cus_group.assign(top_ag=cus_group.apply(lambda x: x['age_gender'][0],
                                                    axis=1))
cus_group.reset_index(inplace=True)

# merge age_gender in purchase data
base = pd.merge(data_pur, cus_group[['cust_no', 'top_ag']], how='left',
                      on='cust_no')
base = base.rename(columns={'top_ag': 'age_gender'})

# merge item categories
base = pd.merge(base, data_pro, how='left', on='item_no')

# split age and gender
base = base.assign(age=base.apply(lambda x: x['age_gender'].split()[0], axis=1))
base = base.assign(gender=base.apply(lambda x: x['age_gender'].split()[1], axis=1))

In [33]:
# [1]

# per gender and item_ctg1, calculate unique number of cust_no and sum of item_amt
data1 = base.groupby(['gender', 'item_ctg1']).agg({'item_amt': 'sum',
                                                   'cust_no': 'nunique'})

# calculate amt per 1 customer
data1 = data1.assign(amt_cust=data1.apply(lambda x: x['item_amt'] / x['cust_no'],
                                          axis=1))

# sort in descending by amt
data1 = data1.sort_values(by=['gender', 'amt_cust'], ascending=[True, False])

# find 2nd(index[1]) items per gender
male = data1.loc['M']['amt_cust'].index[1]
female = data1.loc['F']['amt_cust'].index[1]

print(f'Answer [1] : {male}, {female}')

Answer [1] : 의류, 화장품


In [34]:
# [2] rank

import math

# aggregate by customer with amt sum
data2 = base.groupby('cust_no').agg({'item_amt': 'sum'})

# calculate rank: min means "smaller value for tie"
data2 = data2.assign(rank=data2['item_amt'].rank(method='min', ascending=False))

# total number of customers
no_cust = data2.index.nunique()

# classify rank class
def rankClass(x):
    if x <= no_cust * 0.2:
        return 'top'
    elif x >= no_cust * 0.8:
        return 'bottom'
    else:
        return 'normal'

data2 = data2.assign(rank_class=data2.apply(lambda x: rankClass(x['rank']),
                                            axis=1))

# calculate amt by class
data2 = data2.groupby('rank_class').agg({'item_amt': 'sum'})

top_portion = data2.loc['top']['item_amt'] / data2['item_amt'].sum() * 100
top_over_bottom = data2.loc['top']['item_amt'] / data2.loc['bottom']['item_amt']

print(f'Answer [2] : {math.floor(top_portion)}, {math.floor(top_over_bottom)}')

Answer [2] : 49, 9


In [35]:
# [3] ANOVA, MultiComparison

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison
import math

# create total spend and age per customer
data3 = base.groupby('cust_no').agg({'item_amt': 'sum',
                                     'age': lambda x: list(set(x))})

# classify age_class
def ageClass(x):
    if x == '30대':
        return 'G1'
    elif x == '40대':
        return 'G2'
    else:
        return 'G3'

data3 = data3.assign(age_cls=data3.apply(lambda x: ageClass(x['age'][0]), axis=1))

# perform ANOVA test
result = ols(formula='item_amt ~ C(age_cls)', data=data3).fit()
anova_table = anova_lm(result)
f_val = anova_table['F'].iloc[0]

# perform Duncan Test
comparison = MultiComparison(data3['item_amt'], data3['age_cls'])
print(comparison.tukeyhsd())

# find mean values per age_cls and sort
ave = data3.groupby('age_cls').agg({'item_amt': 'mean'}).sort_values(
                                                 by='item_amt', ascending=False)

# print and analyze result
print(f'age_cls order : {ave.index[0]}, {ave.index[1]}, {ave.index[2]}')
print(f'P-val G2-G3 : 0.001 < 0.05 : Reject H0')
print(f'P-val G3-G1 : 0.9 > 0.05 : Cannot reject H0')

print(f'Answer [3] : {math.floor(f_val)}, G2 > G3 = G1')

    Multiple Comparison of Means - Tukey HSD, FWER=0.05     
group1 group2  meandiff  p-adj    lower      upper    reject
------------------------------------------------------------
    G1     G2  8461.1735 0.001   4117.8207 12804.5262   True
    G1     G3   717.8831   0.9  -3454.2396  4890.0058  False
    G2     G3 -7743.2904 0.001 -12467.8902 -3018.6906   True
------------------------------------------------------------
age_cls order : G2, G3, G1
P-val G2-G3 : 0.001 < 0.05 : Reject H0
P-val G3-G1 : 0.9 > 0.05 : Cannot reject H0
Answer [3] : 11, G2 > G3 = G1


In [36]:
# [4] apriori, association_rules

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# create item_ctg2 list per customer
data4 = base.groupby('cust_no').agg({'item_ctg2': lambda x: list(set(x))})

# count number of category
data4 = data4.assign(cnt=data4.apply(lambda x: len(x['item_ctg2']), axis=1))

# filter by cnt >= 2
data4 = data4[data4['cnt'] >= 2]

# create category combinations in a list
cat_list = list(data4['item_ctg2'])

# make transaction encoder
# creates True / False table (number of samples BY categories)
te = TransactionEncoder()
te_array = te.fit(cat_list).transform(cat_list)
te_df = pd.DataFrame(te_array, columns=te.columns_)

# create frequent items sets by support values
freq_items = apriori(te_df, min_support=0.001, use_colnames=True)

# create association_rules
asso_rules = association_rules(freq_items, metric='confidence',
                               min_threshold=0.001)

# find associations which has '농산물' in antecedents
asso_rules = asso_rules.assign(check=asso_rules.apply(lambda x:
                                                      '농산물' in x['antecedents'],
                                                      axis=1))
asso_rules = asso_rules[asso_rules['check'] == 1]

# sort by Lift descending
asso_rules = asso_rules.sort_values(by='lift', ascending=False)

recomm = list(asso_rules['consequents'].iloc[0])

print(f'Answer [4] : {recomm[0]}')

Answer [4] : 가공식품
